In [236]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

In [237]:
price_chaos_per_ex = 113.0
with open('.env-postgres') as f:
    lines = f.readlines()
engine = create_engine('postgresql://' + lines[0] + '@localhost:5432/poeitems')
df=pd.read_sql_query('select * from items',con=engine)
df.dropna(subset='explicit', axis='rows', inplace=True)

In [238]:
explicit_mod_dict = {'mod':['']}
explicit_mods = pd.DataFrame.from_dict(explicit_mod_dict)
for _, item in df.iterrows():
    for mod in item.explicit:
        mod_no_digits = ''.join([i for i in mod if i.isalpha() or i==' '])
        if mod_no_digits not in explicit_mods.values:
            explicit_mods.loc[len(explicit_mods.index)] = mod_no_digits

explicit_mods.sort_values(by='mod', inplace=True)
explicit_mods['mod'].replace('',np.nan, inplace=True)
explicit_mods.dropna(inplace=True)
print(explicit_mods['mod'].values)

[' Life gained for each Enemy hit by your Attacks'
 ' Life gained for each Enemy hit by your Spells' ' Life gained on Kill'
 ' Mana gained for each Enemy Hit by your Spells'
 ' Mana gained for each Enemy hit by your Attacks' ' Mana gained on Kill'
 ' chance to Impale Enemies on Hit with Attacks'
 ' chance to gain a Frenzy Charge on Kill'
 ' chance to gain a Power Charge on Kill'
 ' chance to gain an Endurance Charge on Kill' ' increased Area of Effect'
 ' increased Area of Effect of Hex Skills' ' increased Armour'
 ' increased Attack Speed'
 ' increased Attack Speed if youve been Hit Recently'
 ' increased Bleeding Duration' ' increased Cast Speed'
 ' increased Chaos Damage' ' increased Cold Damage' ' increased Damage'
 ' increased Damage with Bleeding' ' increased Damage with Poison'
 ' increased Effect of NonDamaging Ailments'
 ' increased Elemental Damage with Attack Skills'
 ' increased Evasion Rating' ' increased Experience gain'
 ' increased Fire Damage' ' increased Global Accura

In [239]:
implicit_mod_dict = {'mod':['']}
implicit_mods = pd.DataFrame.from_dict(implicit_mod_dict)
for _, item in df.iterrows():
    for mod in item.implicit:
        mod_no_digits = ''.join([i for i in mod if i.isalpha() or i==' '])
        if mod_no_digits not in implicit_mods.values:
            implicit_mods.loc[len(implicit_mods.index)] = mod_no_digits

implicit_mods.sort_values(by='mod', inplace=True)
implicit_mods['mod'].replace('',np.nan, inplace=True)
implicit_mods.dropna(inplace=True)
print(implicit_mods)

                                                   mod
117                                 Chaos Damage taken
104                        Cold Damage taken from Hits
48                         Fire Damage taken from Hits
32      Life gained for each Enemy hit by your Attacks
53                                 Life gained on Kill
..                                                 ...
106                        Regenerate  Life per second
37   Regenerate  of Life per second per Endurance C...
17                    Wrath has  increased Aura Effect
108  You and your Minions take  reduced Reflected D...
80                 Zealotry has  increased Aura Effect

[117 rows x 1 columns]


In [240]:
info_list = ['itemid', 'price', 'basetype', 'ilvl']
explicit_list = explicit_mods['mod'].tolist()
implicit_list = implicit_mods['mod'].tolist()
affix_list = explicit_list + implicit_list
affix_list = list(set(affix_list))
df_affix = pd.DataFrame(columns=info_list + affix_list)
print(df_affix.head())

Empty DataFrame
Columns: [itemid, price, basetype, ilvl, Adds  to  Cold Damage against Chilled or Frozen Enemies,  increased Damage while Leeching Mana, Grants Level  Flammability Skill, Adds  to  Lightning Damage to Spells and Attacks,  to Cold and Chaos Resistances,  to Intelligence,  chance to Avoid being Shocked,  to Level of Socketed Gems,  increased total Recovery per second from Life Leech,  increased Mana Regeneration Rate,  to Dexterity,  Lightning Damage taken from Hits,  to Lightning Resistance, Focus has  increased Cooldown Recovery Rate,  of Fire Damage Leeched as Life, Hatred has  increased Aura Effect,  Life gained for each Enemy hit by your Attacks,  increased Life Reservation Efficiency of Skills,  to Fire and Chaos Resistances,  reduced Effect of Curses on you, Bleeding cannot be inflicted on you,  to Armour, Wrath has  increased Aura Effect,  increased Experience gain,  increased Damage with Ailments, Exerted Attacks deal  increased Damage,  reduced Mana Cost of Skil

In [247]:
def affix_parser():
    return {' Mana gained on Kill':['10'],
            ' chance to Impale Enemies on Hit with Attacks':['100'],
            'ilvl':['60']}

df_temp1 = pd.DataFrame.from_dict(affix_parser())
print(df_temp1.head())

df_temp3 = df_affix.merge(df_temp1, how='outer').fillna(0)
print(df_temp3.head())


   Mana gained on Kill  chance to Impale Enemies on Hit with Attacks ilvl
0                   10                                           100   60
